In [157]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles
using Random

In [158]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [159]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity1.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance1.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"),header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File(string(path,"D_Dsoc.csv"),header=1,delim=",") |> DataFrame
emi_sink = CSV.File(string(path,"emission_sink.csv"),header=1,delim=",") |> DataFrame;

In [160]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
penalty = 5; # ton/yr;

In [163]:
scen_eff = CSV.File(string(path,"Scenarios/Scen_Efficiency2.csv"), header=1, delim=",") |> DataFrame
scen_eff = Matrix(scen_eff)[:, 2:end]

scen_d = CSV.File(string(path,"Scenarios/Scen_Demand.csv"), header=1, delim=",") |> DataFrame
scen_dmd = Matrix(scen_d)[2:end]
# scen_dmd = rand(scen_d, 19)


Omega = ones(Float64, ncty, 50, nproc)  # Omega[ncty,nscena,nproc]
Omega[:, :, 1] = scen_eff
num_omega = size(Omega)[2];

---

In [164]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= 0.8*capacity[!, 2:end][i,k])
        end
    end

    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];
    

    ###########
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    
    G = 0
    g = 0
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, sum(x .* G * ones(nproc,1)) + theta >= sum(g * ones(nproc,1))); 
        end
    end
    
    
    @objective(model, Min, sum(proD-proS)+theta)
    JuMP.optimize!(model);
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result
end

masterprob (generic function with 1 method)

---

In [166]:
M = -10000
toler = 0.01

0.01

In [167]:
i = 0
z_ub = 1e8
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2024-12-26


-1.4151397428710328e7

In [168]:
list = [];

In [169]:
for s in 1:50
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, unmeet >= 0)

    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty] 
    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == scen_dmd[s])
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    tranD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
        end
        tranD[j] = arc_emi
    end


    @objective(model, Min, sum(tranD)+unmeet*penalty)
    JuMP.optimize!(model)
    
    if termination_status(model) == MOI.OPTIMAL
        push!(list, s)
    end
end


Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

In [170]:
list

15-element Vector{Any}:
  1
  2
  4
  5
  6
 11
 12
 14
 24
 26
 27
 29
 31
 33
 37

In [165]:
list

15-element Vector{Any}:
  1
  2
  4
  5
  6
 11
 12
 14
 24
 26
 27
 29
 31
 33
 37

In [146]:
eff_df = DataFrame(scen_eff[:, list],:auto)
eff_df[!,:"country"] = countries
eff_df

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,country
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,String
1,0.946939,0.946939,0.522449,0.946939,1.0449,0.522449,0.522449,0.963265,0.587755,0.587755,0.930612,1.02857,0.587755,0.587755,1.0449,Argentina
2,1.30063,1.26412,0.912721,1.26412,0.912721,1.30063,1.02681,1.02681,1.02681,0.319452,1.30063,0.319452,1.26412,0.271535,1.30063,Australia
3,1.54386,0.210526,0.701754,0.0701754,0.140351,0.982456,0.701754,0.140351,0.982456,0.140351,1.05263,0.982456,0.210526,0.140351,0.210526,Brazil
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Canada
5,1.25,0.519231,1.25,1.25,0.923077,0.653846,0.653846,0.923077,0.653846,0.519231,0.519231,0.653846,1.03846,0.519231,1.25,Chile
6,0.677804,0.21957,1.03103,1.33652,1.03103,1.26969,0.190931,0.649165,0.677804,0.677804,1.26969,0.677804,0.21957,0.21957,1.33652,China
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Colombia
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Congo
9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Finland


In [154]:
dmd_df = DataFrame(vector = scen_dmd[list])

Row,vector
,Any
1,380000
2,380000
3,380000
4,635000
5,380000
6,206000
7,206000
8,635000
9,563000


In [155]:
CSV.write(string(path,"FeaEff_15.csv"), eff_df)

"C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/FeaEff_15.csv"

In [156]:
CSV.write(string(path,"FeaDmd_15.csv"), dmd_df)

"C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/FeaDmd_15.csv"